# Feasible Labour Portfolio Choices

Creates a Function which is used to create all feasible future portfolios given a starting Portfolio.  

This Function is used to create all the feasible portfolios for next period given a starting portfolio. This function differs in the $\textit{Portf}$ Function in that the ceiling for workers in tenure $g$ is the current number of workers in tenure $g-1$.

### Input Variables:

$G$ Number of Tenure Groups  
$N$ Max Number of Workers Always available for first year of tenure

### Output Variables:

$\textit{Feasf}$ Function that can Evaluate all Feasable Portfolios

### Function Argument:

$L_i$ Vector with the Maximum Number of Workers for each Tenure Group

## Function:

### Expression:

In [1]:
function eFeas(G::Int64,N::Int64)
    
    """Creates an Expression which is used to create all
    possible combination of portfolios, given a 
    maximum number of workers for each G years of tenure.
    This Function is used to create all possible portfolios 
    in an economy, and the portfolios a portfolio can change 
    to due to a Quit and Retirement shocks.

    Parameters
    ----------
    G    : Number of Tenure Groups
    N    : Max Number of Workers Always available for first year of tenure
    
    Output
    ----------
    Feasf: Expression with Function that can Evaluate all Possible Portfolios
    
    Function Arguments
    ----------
    L_i  : Vector with the Maximum Number of Workers for each Tenure Group

    """
    
    Feasf=Expr(:comprehension);
    
    Feasf.args=Array(Any,G+1);
    
    Feasf.args[1]=Expr(:vcat);
    
    Feasf.args[1].args=Array(Any,G);
    
    Feasf.args[2]=Expr(:(=), :x_1, Expr(:(:),0,N));
    
    Feasf.args[1].args[1]=:x_1;
    
    for i=2:G-1
            
        x_i=symbol("x_$i");
        
        Feasf.args[i+1]=Expr(:(=), x_i, Expr(:(:),0,Expr(:ref,:L_i,i-1)));
        Feasf.args[1].args[i]=x_i;
        
    end
    
    Feasf.args[G+1]=Expr(:(=), :x_G, Expr(:(:),0,Expr(:call,:(+),Expr(:ref,:L_i,G-1),Expr(:ref,:L_i,G))));
    Feasf.args[1].args[G]=:x_G;
    
    return Feasf
    
end

eFeas (generic function with 1 method)

### Function:

In [2]:
function fFeas(G::Int64,N::Int64)
    
    """Creates a Function which is used to create all
    possible combination of portfolios, given a 
    maximum number of workers for each G years of tenure.
    This Function is used to create all possible portfolios 
    in an economy, and the portfolios a portfolio can change 
    to due to a Quit and Retirement shocks.

    Parameters
    ----------
    G    : Number of Tenure Groups
    N    : Max Number of Workers Always available for first year of tenure
    
    Output
    ----------
    Feasf: Function that can Evaluate all Possible Portfolios
    
    Function Arguments
    ----------
    L_i  : Vector with the Maximum Number of Workers for each Tenure Group

    """
    
    Feasf=eFeas(G,N);
    
    Feasf=:(L_i->collect(($Feasf)));
    
    Feasf= @eval $Feasf;
    
    return Feasf
    
end

fFeas (generic function with 1 method)

## Example

### Parameters:

In [3]:
G=3;
N=3;

In [4]:
FunctionFeasible=fFeas(G,N);

#### Output Formula:

In [5]:
using DisplayDP: FuncText
using PyCall
@pyimport IPython.display as IpDisp

ExprFeasible=eFeas(G,N);

Text=FuncText(ExprFeasible,G);

IpDisp.Math(Text)

PyObject <IPython.core.display.Math object>